# Cluster-analysis of neighborhood in Tronto

## 1. Installing Data

In [1]:
!pip install beautifulsoup4 lxml html5lib

In [2]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url,header=0)

In [3]:
df=dfs[0]
df_processing=df[df['Borough']!='Not assigned'].reset_index()
df_processed=df_processing.drop(df_processing.columns[0],1)
df_processed.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [4]:
df_processed.shape

(103, 3)

## 2. Getting latitudes and longitudes

In [5]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [6]:
dfg=pd.read_csv('Geospatial_Coordinates.csv')
dfg.columns=['Postal code','Latitude','Longitude']
dfg_processed=pd.merge(df_processed,dfg,on='Postal code')
dfg_processed.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


## 3. Explore neighbors in Tronto - clustering and mapping

In [7]:
import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium  # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [9]:


CLIENT_ID = '4XKVNDTMG0HKCUESIL3B0XS10R1ZPOMLNKHU2TZORV1GBAMT' 
CLIENT_SECRET = 'VD3O0DHE4TXGG03CHQLNVLGTHVFLGKD2VF42DBI2X0FNITE3' 
VERSION = '20180604'
LIMIT = 30

tronto_venues = getNearbyVenues(names=dfg_processed['Neighborhood'],
                                   latitudes=dfg_processed['Latitude'],
                                   longitudes=dfg_processed['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [10]:
# one hot encoding
tronto_onehot = pd.get_dummies(tronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tronto_onehot['Neighborhood'] = tronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tronto_onehot.columns[-1]] + list(tronto_onehot.columns[:-1])
tronto_onehot = tronto_onehot[fixed_columns]

tronto_onehot.head()


,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
tronto_grouped = tronto_onehot.groupby('Neighborhood').mean().reset_index()
tronto_grouped.head()


,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,Alderwood / Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
num_top_venues = 5

for hood in tronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tronto_grouped[tronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4                Yoga Studio  0.00


----Alderwood / Long Branch----
          venue  freq
0   Pizza Place  0.22
1  Skating Rink  0.11
2      Pharmacy  0.11
3   Coffee Shop  0.11
4          Pool  0.11


----Bathurst Manor / Wilson Heights / Downsview North----
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2    Pizza Place  0.05
3    Gas Station  0.05
4  Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.08
2      Sandwich Place  0.08
3  Italian Restaurant  0.08
4    Greek Restaurant  

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [14]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tronto_grouped['Neighborhood']

for ind in np.arange(tronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,Alderwood / Long Branch,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Pharmacy,Gym,Pub,Pool,Deli / Bodega,Department Store
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Pharmacy,Sandwich Place,Diner,Bridal Shop,Restaurant,Deli / Bodega,Supermarket,Sushi Restaurant
3,Bayview Village,Bank,Chinese Restaurant,Japanese Restaurant,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,Bedford Park / Lawrence Manor East,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Greek Restaurant,Hobby Shop,Pharmacy,Pizza Place,Café,Butcher


In [15]:
# set number of clusters
kclusters = 7

tronto_grouped_clustering = tronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


tronto_merged = dfg_processed

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tronto_merged = tronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tronto_merged.head() # check the last columns!


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,6.0,Park,Pool,Food & Drink Shop,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,French Restaurant,Portuguese Restaurant,Hockey Arena,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,2.0,Coffee Shop,Park,Breakfast Spot,Bakery,Yoga Studio,Mexican Restaurant,Restaurant,Pub,Café,Chocolate Shop
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,2.0,Furniture / Home Store,Clothing Store,Women's Store,Arts & Crafts Store,Event Space,Miscellaneous Shop,Boutique,Coffee Shop,Vietnamese Restaurant,Accessories Store
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,2.0,Coffee Shop,Sushi Restaurant,Diner,Hobby Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Distribution Center,Discount Store,Fried Chicken Joint


In [17]:
tronto_merged2=tronto_merged.dropna(subset=['Cluster Labels'])
tronto_merged2

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,6.0,Park,Pool,Food & Drink Shop,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,French Restaurant,Portuguese Restaurant,Hockey Arena,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,2.0,Coffee Shop,Park,Breakfast Spot,Bakery,Yoga Studio,Mexican Restaurant,Restaurant,Pub,Café,Chocolate Shop
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,2.0,Furniture / Home Store,Clothing Store,Women's Store,Arts & Crafts Store,Event Space,Miscellaneous Shop,Boutique,Coffee Shop,Vietnamese Restaurant,Accessories Store
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,2.0,Coffee Shop,Sushi Restaurant,Diner,Hobby Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Distribution Center,Discount Store,Fried Chicken Joint
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
7,M3B,North York,Don Mills,43.745906,-79.352188,2.0,Restaurant,Coffee Shop,Asian Restaurant,Japanese Restaurant,Gym,Beer Store,Bike Shop,Sporting Goods Shop,Sandwich Place,Dim Sum Restaurant
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,2.0,Pizza Place,Pet Store,Fast Food Restaurant,Bank,Intersection,Café,Athletics & Sports,Gastropub,Pharmacy,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2.0,Café,Coffee Shop,Theater,College Rec Center,Ramen Restaurant,Burger Joint,Fast Food Restaurant,Hotel,Steakhouse,Burrito Place
10,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Park,Pizza Place,Japanese Restaurant,Pub,Metro Station,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center


In [18]:

# create map
latitude=43.651070
longitude=-79.347015
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tronto_merged2['Latitude'], tronto_merged2['Longitude'], tronto_merged2['Neighborhood'], tronto_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster0

In [19]:
tronto_merged2[tronto_merged2['Cluster Labels']==0.0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,French Restaurant,Portuguese Restaurant,Hockey Arena,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
22,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


In [57]:
print(tronto_grouped[tronto_grouped['Neighborhood']=='Victoria Village']['Coffee Shop'])
print(tronto_grouped[tronto_grouped['Neighborhood']=='Woburn']['Coffee Shop'])
tronto_grouped.sort_values('Coffee Shop',ascending=False)[['Coffee Shop','Neighborhood']].head()

85    0.25
Name: Coffee Shop, dtype: float64
91    0.666667
Name: Coffee Shop, dtype: float64


,Coffee Shop,Neighborhood
91,0.666667,Woburn
13,0.300000,Central Bay Street
85,0.250000,Victoria Village
63,0.200000,Regent Park / Harbourfront
62,0.200000,Queen's Park / Ontario Provincial Government


### Cluster1

In [20]:
tronto_merged2[tronto_merged2['Cluster Labels']==1.0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
27,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Fast Food Restaurant,Dog Run,Mediterranean Restaurant,Pool,Dance Studio,Drugstore,Donut Shop,Distribution Center,Discount Store


In [76]:
tronto_grouped.sort_values('Fast Food Restaurant',ascending=False)[['Fast Food Restaurant','Neighborhood']].head()

,Fast Food Restaurant,Neighborhood
49,1.000000,Malvern / Rouge
37,0.200000,Hillcrest Village
74,0.166667,Steeles West / L'Amoreaux West
71,0.111111,South Steeles / Silverstone / Humbergate / Jam...
60,0.090909,Parkview Hill / Woodbine Gardens


### Cluster2

In [21]:
tronto_merged2[tronto_merged2['Cluster Labels']==2.0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,2.0,Coffee Shop,Park,Breakfast Spot,Bakery,Yoga Studio,Mexican Restaurant,Restaurant,Pub,Café,Chocolate Shop
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,2.0,Furniture / Home Store,Clothing Store,Women's Store,Arts & Crafts Store,Event Space,Miscellaneous Shop,Boutique,Coffee Shop,Vietnamese Restaurant,Accessories Store
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,2.0,Coffee Shop,Sushi Restaurant,Diner,Hobby Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Distribution Center,Discount Store,Fried Chicken Joint
7,M3B,North York,Don Mills,43.745906,-79.352188,2.0,Restaurant,Coffee Shop,Asian Restaurant,Japanese Restaurant,Gym,Beer Store,Bike Shop,Sporting Goods Shop,Sandwich Place,Dim Sum Restaurant
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,2.0,Pizza Place,Pet Store,Fast Food Restaurant,Bank,Intersection,Café,Athletics & Sports,Gastropub,Pharmacy,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2.0,Café,Coffee Shop,Theater,College Rec Center,Ramen Restaurant,Burger Joint,Fast Food Restaurant,Hotel,Steakhouse,Burrito Place
10,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Park,Pizza Place,Japanese Restaurant,Pub,Metro Station,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center
12,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,2.0,Bar,History Museum,Women's Store,Department Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
13,M3C,North York,Don Mills,43.725900,-79.340923,2.0,Restaurant,Coffee Shop,Asian Restaurant,Japanese Restaurant,Gym,Beer Store,Bike Shop,Sporting Goods Shop,Sandwich Place,Dim Sum Restaurant
14,M4C,East York,Woodbine Heights,43.695344,-79.318389,2.0,Dance Studio,Cosmetics Shop,Beer Store,Skating Rink,Curling Ice,Park,Pharmacy,Comic Shop,Concert Hall,Eastern European Restaurant


### Cluster3

In [22]:
tronto_merged2[tronto_merged2['Cluster Labels']==3.0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,M2M,North York,Willowdale / Newtonbrook,43.789053,-79.408493,3.0,Piano Bar,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


In [73]:
tronto_grouped.sort_values('Piano Bar',ascending=False)[['Piano Bar','Neighborhood']].head()

,Piano Bar,Neighborhood
90,1.0,Willowdale / Newtonbrook
0,0.0,Agincourt
59,0.0,Parkdale / Roncesvalles
68,0.0,Runnymede / Swansea
67,0.0,Rouge Hill / Port Union / Highland Creek


### Cluster4

In [23]:
tronto_merged2[tronto_merged2['Cluster Labels']==4.0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509,4.0,Baseball Field,Women's Store,Department Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


### Cluster5

In [24]:
tronto_merged2[tronto_merged2['Cluster Labels']==5.0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936,5.0,Garden,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


### Cluster6

In [90]:
tronto_merged2[tronto_merged2['Cluster Labels']==6.0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,6.0,Park,Pool,Food & Drink Shop,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,6.0,Park,Women's Store,Pool,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
35,M4J,East York,East Toronto,43.685347,-79.338106,6.0,Park,Convenience Store,Metro Station,Women's Store,Deli / Bodega,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
49,M6L,North York,North Park / Maple Leaf Park / Upwood Park,43.713756,-79.490074,6.0,Park,Construction & Landscaping,Bakery,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,6.0,Park,Bus Line,Swim School,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
64,M9N,York,Weston,43.706876,-79.518188,6.0,Park,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
66,M2P,North York,York Mills West,43.752758,-79.400049,6.0,Park,Bank,Convenience Store,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
68,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,6.0,Park,Jewelry Store,Sushi Restaurant,Trail,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
83,M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.383160,6.0,Park,Tennis Court,Restaurant,College Rec Center,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
85,M1V,Scarborough,Milliken / Agincourt North / Steeles East / L'...,43.815252,-79.284577,6.0,Park,Gym,Playground,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


##  Summary


Cluster0: More coffee shops are gathered in this area.,
Cluster1: The ratio of fast food resaurant is very high, but the data about neighbourhood in Marveln is not enough.,
Cluster2: As there are many kinds of shops, these area could be big city or business district in Tronto.,
Cluster3: The only venue entried in Willowdale is piano bar. That resulted in the result.,
Cluster4: The unique venue, baseball field, will affect the result.,
Cluster5: The unique venue, garden, will affect the result.,
Cluster6: This area contains a lot of parks and gyms compared to other venues. 